<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-23 15:18:48
-------------------
qualified stocks: 88
with latest results: 28
still star stocks: 17
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  54.95 K
Current:  1.42 C
-------------------
Today PnL: 45.80 K (0.32%)
Current PnL: -19.60 L (-13.06%)
CY Booked + Current PnL: -7.50 L (-5.0%)
-------------------
Total profit:  2.16 L
Total loss:  -21.76 L
-------------------
Total Booked + Current PnL: 19.48 L (15.82%)
Total Booked PnL: 39.08 L (31.74%)
Curr Year Booked PnL: 12.10 L (8.52%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 86.80 L (61.1%)
Deployed:  1.23 C
Current:  1.42 C
CAGR/XIRR %: 8.44%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
58,RELIANCE,1533.0,-9.91,65.0,X-LC,2.99,225654.0,15288.0,13494.0,-1.28,7.27,5.98,13.68,37.0,1.13,1.59,24.99,XY25,NTT,REFINERIES
85,VOLTAS,1530.0,2.28,58.0,X-MC,2.39,214500.0,22758.0,14994.0,-1.77,11.87,6.99,19.69,99.0,1.52,1.52,19.47,XY25,NTT,AC
2,ABBOTINDIA,35195.0,-8.63,47.0,X-MC,3.48,90315.0,-363.0,15272.0,-0.50,-0.40,16.91,16.44,101.0,-0.02,0.64,19.69,X40,ATH,PHARMA
78,TTKPRESTIG,770.0,94.88,47.0,M-SC,1.92,83984.0,-16793.0,16889.0,-0.56,-16.66,20.11,0.09,245.0,-0.99,0.59,10.34,OX40N,NTT,DURABLES
42,ITC,452.0,-37.08,63.0,X-LC,2.14,203448.0,3310.0,17578.0,0.77,1.65,8.64,10.44,4.0,0.19,1.44,7.31,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,BSOFT,836.99,-14.41,62.0,H-SC,15.19,96212.0,-44441.0,117225.0,7.48,-31.60,121.84,51.74,131.0,-0.38,0.68,11.31,XR,ATH,IT
36,IEX,219.00,-31.08,59.0,H-SC,17.60,196643.0,-3311.0,100544.0,4.15,-1.66,51.13,48.63,115.0,-0.03,1.39,9.71,XR,NTT,MISC
12,BANKINDIA,190.00,-19.25,73.0,H-MC,13.67,208414.0,28606.0,83803.0,4.03,15.91,40.21,62.52,88.0,0.34,1.47,55.42,XR,NTT,BANKS
82,VAIBHAVGBL,521.00,51.52,57.0,H-SC,4.68,131604.0,-51171.0,162241.0,3.73,-28.00,123.28,60.77,125.0,-0.32,0.93,17.87,XR,NTT,JEWELLERY
40,INFY,2275.00,-16.37,59.0,X-LC,7.39,325187.0,12299.0,159374.0,3.69,3.93,49.01,54.87,3.0,0.08,2.30,10.84,X40,BTT,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,RAJOOENG,143.10,-42.01,45.0,H-SC,18.22,87900.0,-14600.0,55201.0,-3.24,-14.24,62.80,39.61,114.0,-0.26,0.62,7.35,AR,ATH,MISC
72,TANLA,1963.11,-28.95,34.0,H-SC,17.09,190913.0,-79848.0,409795.0,-2.13,-29.49,214.65,121.86,133.0,-0.19,1.35,49.35,AR,ATH,IT
20,COFFEEDAY,80.00,-45.67,49.0,L-SC,26.64,81120.0,-32429.0,72481.0,-1.86,-28.56,89.35,35.27,268.0,-0.45,0.57,98.54,XR,NTT,HOTELS
85,VOLTAS,1530.00,2.28,58.0,X-MC,2.39,214500.0,22758.0,14994.0,-1.77,11.87,6.99,19.69,99.0,1.52,1.52,19.47,XY25,NTT,AC
71,SYMPHONY,1306.00,-43.99,48.0,M-SC,18.64,119262.0,-51879.0,51819.0,-1.76,-30.31,43.45,-0.03,196.0,-1.00,0.84,3.97,OX40N,NTT,DURABLES


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAIL,228.00,41.14,46.0,M-MC,10.28,222878.0,-2084.0,169744.0,-0.40,-0.93,76.16,74.53,192.0,-0.01,1.57,31.03,XY24,BTT,STEEL
37,INDIAMART,4810.62,-55.39,42.0,H-SC,7.36,124460.0,1124.0,130509.0,-1.19,0.91,104.86,106.72,119.0,0.01,0.88,24.50,AR,ATH,MISC
87,WIPRO,420.00,-13.79,48.0,M-LC,6.28,151367.0,422.0,109030.0,1.15,0.28,72.03,72.51,53.0,0.00,1.07,6.32,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,UJJIVANSFB,60.0,121.70,68.0,H-SC,14.43,140211.0,-2268.0,21789.0,3.36,-1.59,15.54,13.70,163.0,-0.10,0.99,65.49,OX40N,NTT,BANKS
45,KANSAINER,340.0,-66.69,61.0,H-SC,2.19,229293.0,-40374.0,76699.0,-0.81,-14.97,33.45,13.47,138.0,-0.53,1.62,16.59,XY24,NTT,PAINTS
78,TTKPRESTIG,770.0,94.88,47.0,M-SC,1.92,83984.0,-16793.0,16889.0,-0.56,-16.66,20.11,0.09,245.0,-0.99,0.59,10.34,OX40N,NTT,DURABLES
67,SIS,528.0,2078.76,54.0,H-SC,3.43,87937.0,-23095.0,46703.0,0.09,-20.80,53.11,21.26,156.0,-0.49,0.62,18.81,OX40N,NTT,MISC
73,TATAELXSI,9161.0,-23.04,52.0,H-MC,7.60,104063.0,-27772.0,69993.0,2.29,-21.07,67.26,32.03,98.0,-0.40,0.74,16.56,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RECLTD,446.00,45.36,49.0,M-LC,5.86,206030.0,2860.0,39269.0,-0.68,1.41,19.06,20.74,55.0,0.07,1.46,7.03,XY25,NTT,FINANCE
32,HINDZINC,730.22,28.52,50.0,M-LC,10.61,209322.0,4246.0,108324.0,-0.55,2.07,51.75,54.89,52.0,0.04,1.48,26.66,X5K,ATH,METALS
46,KPIGREEN,731.05,7.10,68.0,H-SC,5.26,127663.0,2366.0,56567.0,0.30,1.89,44.31,47.03,141.0,0.04,0.90,59.85,MH,ATH,POWER
80,UNIONBANK,163.00,-10.08,61.0,M-LC,8.99,162489.0,21649.0,22846.0,0.88,15.37,14.06,31.59,66.0,0.95,1.15,45.93,XY24,NTT,BANKS
87,WIPRO,420.00,-13.79,48.0,M-LC,6.28,151367.0,422.0,109030.0,1.15,0.28,72.03,72.51,53.0,0.00,1.07,6.32,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,190.00,-19.25,73.0,H-MC,13.67,208414.0,28606.0,83803.0,4.03,15.91,40.21,62.52,88.0,0.34,1.47,55.42,XR,NTT,BANKS
86,WHIRLPOOL,2270.00,-37.89,68.0,M-SC,3.97,105030.0,13532.0,65224.0,0.21,14.79,62.10,86.07,223.0,0.21,0.74,52.52,XR,NTT,DURABLES
46,KPIGREEN,731.05,7.10,68.0,H-SC,5.26,127663.0,2366.0,56567.0,0.30,1.89,44.31,47.03,141.0,0.04,0.90,59.85,MH,ATH,POWER
32,HINDZINC,730.22,28.52,50.0,M-LC,10.61,209322.0,4246.0,108324.0,-0.55,2.07,51.75,54.89,52.0,0.04,1.48,26.66,X5K,ATH,METALS
37,INDIAMART,4810.62,-55.39,42.0,H-SC,7.36,124460.0,1124.0,130509.0,-1.19,0.91,104.86,106.72,119.0,0.01,0.88,24.50,AR,ATH,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TATAMOTORS,1065.00,-53.17,16.0,X-LC,13.41,147124.0,-129399.0,239474.0,0.86,-46.79,162.77,39.81,54.0,-0.54,1.04,3.70,XY24,NTT,AUTO
53,QUESS,986.00,-48.21,35.0,X-SC,38.78,51410.0,-13596.0,164522.0,-1.68,-20.91,320.02,232.18,198.0,-0.08,0.36,0.28,XY24,NTT,MISC
24,DMART,5391.45,-16.17,37.0,X-LC,10.17,102518.0,-68.0,26880.0,-0.07,-0.07,26.22,26.13,38.0,-0.00,0.72,25.49,X40N,ATH,FMCG
51,PAGEIND,51769.93,-26.69,43.0,X-MC,6.71,82820.0,-690.0,20722.0,-0.42,-0.83,25.02,23.98,82.0,-0.03,0.59,4.95,X40,ATH,APPARELS
52,PGHH,17907.65,-28.96,45.0,X-MC,3.29,205545.0,4725.0,63061.0,-1.20,2.35,30.68,33.76,80.0,0.07,1.45,7.67,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
66,SIEMENS,4671.50,-1.97,49.0,H-LC,0.67,157450.0,-28645.0,76127.0,1.49,-15.39,48.35,25.51,15.0,-0.38,1.11,16.03,AR,ATH,ELECTRICAL
42,ITC,452.00,-37.08,63.0,X-LC,2.14,203448.0,3310.0,17578.0,0.77,1.65,8.64,10.44,4.0,0.19,1.44,7.31,X40,NTT,FMCG
85,VOLTAS,1530.00,2.28,58.0,X-MC,2.39,214500.0,22758.0,14994.0,-1.77,11.87,6.99,19.69,99.0,1.52,1.52,19.47,XY25,NTT,AC
29,HAVELLS,2069.16,-6.18,45.0,X-MC,2.82,232690.0,-15041.0,90098.0,0.51,-6.07,38.72,30.30,92.0,-0.17,1.64,6.41,X40,ATH,ELECTRICAL
58,RELIANCE,1533.00,-9.91,65.0,X-LC,2.99,225654.0,15288.0,13494.0,-1.28,7.27,5.98,13.68,37.0,1.13,1.59,24.99,XY25,NTT,REFINERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-48.21,35.0,X-SC,38.78,51410.0,-13596.0,164522.0,-1.68,-20.91,320.02,232.18,198.0,-0.08,0.36,0.28,XY24,NTT,MISC
74,TATAMOTORS,1065.00,-53.17,16.0,X-LC,13.41,147124.0,-129399.0,239474.0,0.86,-46.79,162.77,39.81,54.0,-0.54,1.04,3.70,XY24,NTT,AUTO
3,ACC,3906.00,-37.64,51.0,X-MC,3.02,185220.0,-52731.0,205372.0,0.18,-22.16,110.88,64.15,174.0,-0.26,1.31,3.90,XY24,BTT,CEMENT
10,BAJAJHFL,181.50,-14.27,47.0,X-MC,5.68,180613.0,-19893.0,117055.0,-0.52,-9.92,64.81,48.45,90.0,-0.17,1.28,4.15,X40N,ATH,FINANCE
51,PAGEIND,51769.93,-26.69,43.0,X-MC,6.71,82820.0,-690.0,20722.0,-0.42,-0.83,25.02,23.98,82.0,-0.03,0.59,4.95,X40,ATH,APPARELS


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-48.21,35.0,X-SC,38.78,51410.0,-13596.0,164522.0,-1.68,-20.91,320.02,232.18,198.0,-0.08,0.36,0.28,XY24,NTT,MISC
51,PAGEIND,51769.93,-26.69,43.0,X-MC,6.71,82820.0,-690.0,20722.0,-0.42,-0.83,25.02,23.98,82.0,-0.03,0.59,4.95,X40,ATH,APPARELS
57,RELAXO,1176.00,-38.81,47.0,X-SC,4.74,83387.0,-61773.0,137697.0,0.21,-42.56,165.13,52.31,136.0,-0.45,0.59,11.62,X40N,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-8.63,47.0,X-MC,3.48,90315.0,-363.0,15272.0,-0.50,-0.40,16.91,16.44,101.0,-0.02,0.64,19.69,X40,ATH,PHARMA
13,BATAINDIA,2096.00,-35.01,48.0,X-SC,4.31,97110.0,-31560.0,76863.0,0.39,-24.53,79.15,35.20,219.0,-0.41,0.69,11.46,X40,NTT,FOOTWEAR


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.96,-27.40,58.0,X-LC,12.52,288759.0,-57197.0,123906.0,2.17,-16.53,42.91,19.28,1.0,-0.46,2.04,6.75,X40,ATH,IT
40,INFY,2275.00,-16.37,59.0,X-LC,7.39,325187.0,12299.0,159374.0,3.69,3.93,49.01,54.87,3.0,0.08,2.30,10.84,X40,BTT,IT
42,ITC,452.00,-37.08,63.0,X-LC,2.14,203448.0,3310.0,17578.0,0.77,1.65,8.64,10.44,4.0,0.19,1.44,7.31,X40,NTT,FMCG
84,VBL,671.64,-17.63,56.0,X-LC,10.15,296734.0,-19108.0,131780.0,1.23,-6.05,44.41,35.67,5.0,-0.14,2.10,6.97,X40N,ATH,FMCG
1,ABB,7934.00,-40.55,47.0,H-LC,6.50,248616.0,-13003.0,132214.0,-1.15,-4.97,53.18,45.57,7.0,-0.10,1.76,5.79,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,137.00,7250.00,71.0,L-SC,7.01,80673.0,-13137.0,89757.0,0.59,-14.00,111.26,81.67,269.0,-0.15,0.57,56.23,XR,NTT,CERAMICS
20,COFFEEDAY,80.00,-45.67,49.0,L-SC,26.64,81120.0,-32429.0,72481.0,-1.86,-28.56,89.35,35.27,268.0,-0.45,0.57,98.54,XR,NTT,HOTELS
49,MASFIN,398.61,-19.15,51.0,H-SC,9.52,92100.0,-5880.0,27483.0,-0.36,-6.00,29.84,22.05,152.0,-0.21,0.65,33.87,XR,ATH,FINANCE
86,WHIRLPOOL,2270.00,-37.89,68.0,M-SC,3.97,105030.0,13532.0,65224.0,0.21,14.79,62.10,86.07,223.0,0.21,0.74,52.52,XR,NTT,DURABLES
46,KPIGREEN,731.05,7.10,68.0,H-SC,5.26,127663.0,2366.0,56567.0,0.30,1.89,44.31,47.03,141.0,0.04,0.90,59.85,MH,ATH,POWER


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,SAMMAANCAP,326.00,-190.91,74.0,M-SC,22.59,156780.0,7560.0,136618.0,-0.53,5.07,87.14,96.62,208.0,0.06,1.11,69.67,XY25,NTT,FINANCE
79,UJJIVANSFB,60.00,121.70,68.0,H-SC,14.43,140211.0,-2268.0,21789.0,3.36,-1.59,15.54,13.70,163.0,-0.10,0.99,65.49,OX40N,NTT,BANKS
68,SONACOMS,806.63,-32.86,70.0,M-SC,10.25,83528.0,-17631.0,57634.0,3.32,-17.43,69.00,39.54,202.0,-0.31,0.59,17.78,AR,ATH,AUTO
5,ANGELONE,3033.00,16.33,70.0,X-SC,7.05,205574.0,14568.0,43129.0,0.37,7.63,20.98,30.21,157.0,0.34,1.45,30.10,X40N,NTT,FINANCE
12,BANKINDIA,190.00,-19.25,73.0,H-MC,13.67,208414.0,28606.0,83803.0,4.03,15.91,40.21,62.52,88.0,0.34,1.47,55.42,XR,NTT,BANKS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.48
1,25,43.83
2,50,74.75


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.37
LC    32.78
MC    22.84
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.31
X40      16.05
XY25     12.26
XR       11.28
X40N      9.32
AR        7.91
OX40N     7.61
X200      1.78
X5K       1.48
SR        1.09
MH        0.90
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.55
X-LC    21.94
X-MC    17.10
M-SC    12.71
M-LC     5.16
H-LC     4.65
X-SC     4.63
H-MC     3.86
M-MC     1.57
L-SC     1.48
L-LC     1.03
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.76,-3.09,36.73
IT,12.75,-16.57,79.49
FINANCE,12.12,-10.29,60.29
MISC,6.93,-17.18,82.11
BANKS,6.61,-6.67,62.78
PAINTS,5.64,-15.68,32.92
ELECTRICAL,5.28,-10.12,48.75
INSURANCE,3.91,-0.09,37.41
AUTO,2.81,-44.57,104.91


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3284511.0,22
XR,1312200.0,14
AR,1175302.0,9
X40,775791.0,12
X40N,610836.0,8
OX40N,568067.0,10
XY25,448150.0,8
SR,266888.0,2
X5K,108324.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3382621.0,25
M-SC,1208413.0,15
X-MC,1115768.0,13
X-LC,1033836.0,13
X-SC,525859.0,6
H-MC,333420.0,3
H-LC,281230.0,3
M-LC,279469.0,4
L-SC,250782.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1167762.0      6
           AR         843041.0      5
           XR         784684.0      7
M-SC       XY24       688858.0      6
X-MC       XY24       584651.0      4
X-LC       X40        448278.0      5
           XY24       306504.0      2
X-SC       X40N       284474.0      4
H-SC       SR         266888.0      2
           OX40N      263679.0      4
X-MC       X40        250650.0      6
X-LC       X40N       209307.0      3
H-LC       AR         208341.0      2
H-MC       XY24       179624.0      1
M-MC       XY24       169744.0      1
X-SC       XY24       164522.0      1
X-MC       XY25       163412.0      2
L-SC       XR         162238.0      2
M-SC       OX40N      145851.0      4
           XY25       136618.0      1
           AR         123920.0      2
X-MC       X40N       117055.0      1
M-SC       XR         113166.0      2
M-LC       XR         109030.0      1
           X5K        108324.0      1
L-SC       OX40N       88544.0      1
H-MC       XR          83803.0      1
X-SC       X40         76863.0      1
H-LC       X200        72889.0      1
H-MC       OX40N       69993.0      1
X-LC       XY25        69747.0      3
L-MC       XR          59279.0      1
H-SC       MH          56567.0      1
M-LC       XY25        39269.0      1
L-LC       XY25        39104.0      1
M-LC       XY24        22846.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
